In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, random, pandas

In [6]:
driver = webdriver.Chrome()
driver.get(r'https://github.com/')

In [ ]:
# query = 'Twilio'
# search = driver.find_element_by_name('q')
# search.send_keys(query)
# search.send_keys(Keys.RETURN)
time.sleep(5)

result_links = []
links_set = []

output={}
folder_name = ""
output_all = pandas.DataFrame()
results_xpath='//*[@id="js-pjax-container"]//a[contains(@class,"v-align-middle")]'

page_numbers= range(51,52)
for page_number in page_numbers:
    page = "https://github.com/search?p=" + str(page_number) + "&q=Twilio&type=Repositories&utf8=%E2%9C%93"
    driver.get(page)
    time.sleep(3)
    result_links=[]
    try:
        links_set = [i.get_attribute('href') for i in driver.find_elements_by_xpath(results_xpath) if i.get_attribute('href')]
        result_links.extend(links_set)
        print(len(result_links))

        for result_link in result_links[:10]:
            commit_set =[]
            try:
                driver.get(result_link)
                folder_class = driver.find_element_by_class_name('public')
                folder_tag = folder_class.find_element_by_tag_name('strong')
                folder_name = folder_tag.find_element_by_tag_name('a').text
                author_name = driver.find_element_by_class_name('url').text
                
                folder_name = author_name+ "/" +folder_name
                
            except:
                print('extraction failed',result_link)
                continue
            time.sleep(3)        

            commit_link = result_link+"/commits/master"
            driver.get(commit_link)
            time.sleep(3)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)

            inner_pagination_counter = True

            #Get the commit time
            commit_li_elems = []
            commit_list = []
            final_commit = []

            #Loop to get the full list of commits
            while inner_pagination_counter:          
                try:
                    commitlistings = driver.find_element_by_class_name('commits-listing')
                    commit_li_elems = commitlistings.find_elements_by_tag_name('li')
                    commit_list =[commit.find_element_by_tag_name('relative-time').get_attribute('title') for commit in commit_li_elems]
                    final_commit.extend(commit_list)

                    #Click the Older button in Commit Page
                    driver.find_element_by_class_name('pagination').click()
                    time.sleep(5)
                    driver.find_element_by_tag_name('body').send_keys(Keys.END)
                except:
                    inner_pagination_counter = False

            output['folderName'] = folder_name
            output['commitTime'] = final_commit            
            temp = pandas.DataFrame(output)            
            output_all = output_all.append(temp)
    except:
        print("Failed")
    
output_all.to_csv('Twilio.csv')

10
